# Sea ice diagnostics

The most epic sea-ice diagnostics ever!

In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cmocean 

In [2]:
from dask.distributed import Client, LocalCluster, wait
cluster = LocalCluster(n_workers=8, threads_per_worker=2, memory_limit='16GB',
                       silence_logs=50)
client = Client(address=cluster)
client

/gxfs_home/geomar/smomw352/miniconda3/envs/py3_std/lib/python3.7/site-packages/distributed/node.py:164: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 36235 instead
  expected, actual


Client Scheduler: tcp://127.0.0.1:36711 Dashboard: http://127.0.0.1:36235/status,Cluster Workers: 8 Cores: 16 Memory: 119.21 GiB


In [3]:
## Directories

In [4]:
machine = 'nesh'

if machine == 'sci':
    esmdir = '/data/user/jkjellsson/esm-experiments/focioifs/'
elif machine == 'nesh':
    esmdir = '/gxfs_work1/geomar/smomw352/esm-experiments/'
    esmdir2 = '/gxfs_work1/geomar/smomw352/esm-slask/'

In [5]:
## Read NEMO mesh

In [6]:
if machine == 'nesh':
    mesh05 = '/gxfs_work1/geomar/smomw352/orca05_mesh_mask.nc'
    
ds = xr.open_dataset(mesh05)
lat_05 = ds['nav_lat'][:,:]
areacello_05 = ds['e1t'][0,:,:] * ds['e2t'][0,:,:]

In [7]:
## Read sea ice from LIM

In [8]:
#expnames = ['FOCI_GJK001','FOCI_GJK006','FOCI_GJK016','FOCI_GJK016','FOCI_GJK016']
#times_list = [slice('1850-01-01','2850-01-01'),
#              slice('2850-01-01','4000-01-01'),
#              slice('4000-01-01','4100-01-01'), 
#              slice('4100-01-01','4200-01-01'),
#              slice('4200-01-01','4400-01-01')]
#explabel = ['GJK001','GJK006','GJK016','Ocean alb.','Ocean and ice alb']
expnames = ['FOCI_GJK024']
times_list = [slice('1950-01-01','2500-01-01')]
explabel = ['spinup']

In [9]:
ice_all = []

freq = '1m'
grid = 'icemod'

for i, (exp,time) in enumerate( zip(expnames, times_list) ):
    
    time_str = '*' 
    chunks = {'x':None, 'y':None, 'time_counter': 1} 
    
    ddir = '/gxfs_work1/geomar/smomw352/esm-experiments/%s/outdata/nemo/' % (exp,)
    
    # files 
    files = '%s/%s_%s_%s_%s.nc' % (ddir,exp,freq,time_str,grid)
    print(files)
    _ds = xr.open_mfdataset(files,combine='nested',
                            use_cftime=True,
                            concat_dim="time_counter", chunks=chunks,
                            data_vars='minimal', coords='minimal', compat='override',
                            parallel=True).rename({'time_counter':'time', 'nav_lat':'lat', 'nav_lon':'lon'}).sel(time=time)
    
    ice_all.append(_ds)
    

/gxfs_work1/geomar/smomw352/esm-experiments/FOCI_GJK024/outdata/nemo//FOCI_GJK024_1m_*_icemod.nc


In [10]:
_ds

<xarray.Dataset>
Dimensions:               (y: 511, x: 722, time: 3000, axis_nbounds: 2)
Coordinates:
    lat                   (y, x) float32 dask.array<chunksize=(511, 722), meta=np.ndarray>
    lon                   (y, x) float32 dask.array<chunksize=(511, 722), meta=np.ndarray>
    time_centered         (time) object dask.array<chunksize=(1,), meta=np.ndarray>
  * time                  (time) object 1950-01-16 12:00:00 ... 2199-12-16 12...
Dimensions without coordinates: y, x, axis_nbounds
Data variables: (12/16)
    time_centered_bounds  (time, axis_nbounds) object dask.array<chunksize=(1, 2), meta=np.ndarray>
    time_counter_bounds   (time, axis_nbounds) object dask.array<chunksize=(1, 2), meta=np.ndarray>
    ice_pres              (time, y, x) float32 dask.array<chunksize=(1, 511, 722), meta=np.ndarray>
    ileadfra              (time, y, x) float32 dask.array<chunksize=(1, 511, 722), meta=np.ndarray>
    isnowthi              (time, y, x) float32 dask.array<chunksize=(1, 511, 722), meta=np.ndarray>
    iicethic              (time, y, x) float32 dask.array<chunksize=(1, 511, 722), meta=np.ndarray>
    ...                    ...
    iicevelv              (time, y, x) float32 dask.array<chunksize=(1, 511, 722), meta=np.ndarray>
    iicestru              (time, y, x) float32 dask.array<chunksize=(1, 511, 722), meta=np.ndarray>
    iicestrv              (time, y, x) float32 dask.array<chunksize=(1, 511, 722), meta=np.ndarray>
    iicesflx              (time, y, x) float32 dask.array<chunksize=(1, 511, 722), meta=np.ndarray>
    iicenflx              (time, y, x) float64 dask.array<chunksize=(1, 511, 722), meta=np.ndarray>
    isnowpre              (time, y, x) float32 dask.array<chunksize=(1, 511, 722), meta=np.ndarray>
Attributes:
    name:         ../../outdata/nemo/FOCI_GJK024_1m_19500101_19591231_icemod
    description:  ice variables
    title:        ice variables
    Conventions:  CF-1.6
    timeStamp:    2023-Jan-25 14:34:12 GMT
    uuid:         c1620e76-49c2-4fe3-9db0-116c153fdefd
    history:      Thu Jan 26 02:44:59 2023: ncks -7 --no-alphabetize -L 1 --c...
    NCO:          netCDF Operators version 4.7.9 (Homepage = http://nco.sf.ne...

In [11]:
## Compute Arctic and Antarctic totals

In [12]:
def ice_areas(ds):
    
    # ice area = icefrac * cell area
    _nh = (ds['ileadfra'].where(lat_05 > 0) * areacello_05).sum(('x','y')) 
    _sh = (ds['ileadfra'].where(lat_05 < 0) * areacello_05).sum(('x','y')) 
    
    # ice extent = cell area where icefrac > 0.15
    _nh2 = (ds['ileadfra'].where(lat_05 > 0).where(ds['ileadfra'] > 0.15) * areacello_05).sum(('x','y')) 
    _sh2 = (ds['ileadfra'].where(lat_05 < 0).where(ds['ileadfra'] > 0.15) * areacello_05).sum(('x','y')) 
    
    _nh.name = 'ar_sia'
    _sh.name = 'an_sia'
    _nh2.name = 'ar_sie'
    _sh2.name = 'an_sie'
    _ds_n = _nh.to_dataset()
    _ds_s = _sh.to_dataset()
    _ds_n2 = _nh2.to_dataset()
    _ds_s2 = _sh2.to_dataset()
    _ds = xr.merge([_ds_n, _ds_s, _ds_n2, _ds_s2])
    
    return _ds


def ice_volumes(ds):
    
    # iicethic is cell average ice thickness
    # i.e. if half cell is covered by 2m thick ice, iicethic is 1m
    _nh = (ds['iicethic'].where(lat_05 > 0) * areacello_05).sum(('x','y')) 
    _sh = (ds['iicethic'].where(lat_05 < 0) * areacello_05).sum(('x','y')) 
    
    _nh.name = 'ar_siv'
    _sh.name = 'an_siv'
    _ds_n = _nh.to_dataset()
    _ds_s = _sh.to_dataset()
    _ds = xr.merge([_ds_n, _ds_s])
    
    return _ds

In [ ]:
sia_list = []
for dice in ice_all:
    _ds = ice_areas(dice).compute()
    sia_list.append(_ds)

In [ ]:
siv_list = []
for dice in ice_all:
    _ds = ice_volumes(dice).compute()
    siv_list.append(_ds)

In [ ]:
## Plot all sea ice areas

In [ ]:
fig1, ax1 = plt.subplots(1,2, figsize=(12,4))

for i, sia in enumerate(sia_list):
    _m = sia.groupby('time.month').mean('time') * 1e-12
    _m['ar_sie'].plot(ax=ax1[0], label=explabel[i]) 
    _m['an_sie'].plot(ax=ax1[1])

# Draw Feb and Sep climate (1981-2010) from NSIDC
ax1[0].axhline(y=15.30, linestyle='dashed', linewidth=1, c='k')
ax1[0].axhline(y=6.41, linestyle='dotted', linewidth=1, c='k') 

ax1[1].axhline(y=3.07, linestyle='dashed', linewidth=1, c='k')
ax1[1].axhline(y=18.49, linestyle='dotted', linewidth=1, c='k')

ax1[0].set_title('Arctic SIE cycle')
ax1[0].legend()
ax1[1].set_title('Antarctic SIE cycle')

for ax in ax1:
    ax.set_ylabel('[million km2]')
    ax.set_xticks(np.arange(1,13))
    ax.set_xticklabels(['J','F','M','A','M','J','J','A','S','O','N','D'])

In [ ]:
fig1, axs1 = plt.subplots(2,2, figsize=(12,8))
ax1 = axs1.flat

for i, sia in enumerate(sia_list):
    
    # select February
    _f = sia.sel(time=sia.time.dt.month.isin([2])) * 1e-12
    _f['ar_sie'].plot(ax=ax1[0], label=explabel[i])
    _f['an_sie'].plot(ax=ax1[1])
    
    # select September
    _s = sia.sel(time=sia.time.dt.month.isin([9])) * 1e-12
    _s['ar_sie'].plot(ax=ax1[2])
    _s['an_sie'].plot(ax=ax1[3])
    
ax1[0].legend()
ax1[0].set_title('Arctic SIE (Feb)')
ax1[1].set_title('Antarctic SIE (Feb)')
ax1[2].set_title('Arctic SIE (Sep)')
ax1[3].set_title('Antarctic SIE (Sep)')

# Draw climate 1981-2010 from NSIDC
# https://nsidc.org/arcticseaicenews/sea-ice-tools/
ax1[0].axhline(y=15.30, linestyle='dashed', linewidth=1, c='k')
ax1[1].axhline(y=3.07, linestyle='dashed', linewidth=1, c='k')
ax1[2].axhline(y=6.41, linestyle='dashed', linewidth=1, c='k') 
ax1[3].axhline(y=18.49, linestyle='dashed', linewidth=1, c='k')

for ax in ax1:
    ax.set_ylabel('[million km2]')
    
fig1.tight_layout()
fig1.savefig('plots/feb_and_sep_SIE.pdf',format='pdf')